In [1]:
import sys
sys.path.insert(0, "../")

import pandas as pd
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from pathlib import Path
import re
import ast

import os

In [2]:
import scipy

In [3]:
df = pd.read_excel(r'dataset.xlsx')
df

,cuisine,Numbers of order,Customer_id
0,Fast Food,10,1
1,Cakes & Bakery,2,1
2,Western,1,1
3,Non-alcoholic Drinks,1,1
4,Chinese,4,1
...,...,...,...
934,Singaporean,4,100
935,Chicken,2,100
936,Indian,3,100
937,Sandwiches,2,100


# EDA

In [4]:
df.cuisine.unique()

array(['Fast Food', 'Cakes & Bakery', 'Western', 'Non-alcoholic Drinks',
       'Chinese', 'Pizza', 'Thai', 'Indian', 'American', 'Vietnamese',
       'Singaporean', 'Desserts', 'Japanese', 'Burgers', 'Malaysian',
       'Halal', 'Italian', 'Healthy Food', 'European', 'Chicken',
       'Korean', 'Melaka Portuguese', 'Asian', 'Mexican', 'Sandwiches',
       'Indonesian', 'International', 'French', 'Seafood',
       'Middle Eastern', 'Malay', 'Vegetarian', 'Filipino', 'Turkish',
       'German', 'Sushi', 'Mediterranean'], dtype=object)

In [5]:
df.Customer_id.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int64)

In [6]:
df

,cuisine,Numbers of order,Customer_id
0,Fast Food,10,1
1,Cakes & Bakery,2,1
2,Western,1,1
3,Non-alcoholic Drinks,1,1
4,Chinese,4,1
...,...,...,...
934,Singaporean,4,100
935,Chicken,2,100
936,Indian,3,100
937,Sandwiches,2,100


In [7]:
df_pivot = pd.pivot_table(df,index=['Customer_id'],columns=['cuisine'],fill_value = 0,values = "Numbers of order").reset_index()
df_pivot = df_pivot.set_index('Customer_id')
df_pivot 

cuisine,American,Asian,Burgers,Cakes & Bakery,Chicken,Chinese,Desserts,European,Fast Food,Filipino,...,Pizza,Sandwiches,Seafood,Singaporean,Sushi,Thai,Turkish,Vegetarian,Vietnamese,Western
Customer_id,,,,,,,,,,,,,,,,,,,,,
1,2,0,0,2,0,4,1,0,10,0,...,5,0,0,7,0,1,0,0,2,1
2,6,0,6,0,0,11,0,0,2,0,...,2,0,0,14,0,0,0,0,1,4
3,1,0,5,2,1,6,1,1,0,0,...,0,0,0,3,0,2,0,0,3,2
4,2,0,0,1,0,6,0,0,3,0,...,0,0,0,11,0,1,0,0,0,1
5,4,2,0,0,1,4,2,0,2,0,...,0,0,0,3,0,0,0,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2,0,0,0,2,25,0,0,7,0,...,0,0,0,7,0,0,7,0,0,0
97,1,0,1,0,1,6,0,0,0,0,...,1,0,0,0,0,0,0,0,0,8
98,0,0,0,0,0,12,6,0,2,0,...,2,0,0,3,0,0,0,0,0,2


In [8]:
import numpy as np

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import operator
def find_similar_users(user_id, matrix, k=3): 
    """
    The goal is to find 3 most similar users to our target 
    """
    
    # Split matrix with just the target and other users. 
    user = matrix[matrix.index == user_id]
    other_users = matrix[matrix.index != user_id]
    
#     print("Other matrix ", other_users)
    
    # Compute cosine similarity for the target compared to other users.
    similarities = cosine_similarity(user,other_users)[0].tolist()
    

    indices = other_users.index.tolist()
    # create key/values dictionary of other users and their similarity
    index_similarity = dict(zip(indices, similarities))
    
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]
    
#     print("Users ", users)
    return users
    
# Example pick sample user
current_user = 98

similar_user_indices = find_similar_users(current_user, df_pivot)
print(similar_user_indices)

[8, 86, 82]


In [56]:
# Cross checking to see what target has ordered 
df.loc[df.Customer_id == 98]

,cuisine,Numbers of order,Customer_id
914,Fast Food,2,98
915,Singaporean,3,98
916,Pizza,2,98
917,Chinese,12,98
918,Desserts,6,98
919,Western,2,98
920,Korean,9,98
921,Non-alcoholic Drinks,1,98


In [14]:
def recommend_item(user_index, similar_user_indices, matrix, items=3):
    
    # get the matrix with the 3 similar users 
    similar_users = matrix[matrix.index.isin(similar_user_indices)]
    
    # calc avg score across the 3 similar users
    similar_users = similar_users.mean(axis=0)
    
    similar_users_df = pd.DataFrame(similar_users, columns=['mean'])
    
    # get the matrix for target 
    user_df = matrix[matrix.index == user_index]
    user_df_transposed = user_df.transpose()    
    user_df_transposed.columns = ['score']
    
    # extract any cuisines the target user has not tried 
    user_df_transposed = user_df_transposed[user_df_transposed['score']==0]
    
    cuisines_nottried = user_df_transposed.index.tolist()
        
    # Retrieve cuisines from similar users where cuisines target has not tried 
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(cuisines_nottried)]    
    similar_users_df_filtered = similar_users_df_filtered.sort_values(by=['mean'], ascending=False)
        
     
    top_n_food = similar_users_df_filtered.head(items)
    top_n_food_indices = top_n_food.index.tolist()
    
    return top_n_food_indices 


top3 = recommend_item(current_user, similar_user_indices, df_pivot)
print("Try out these top 3 cuisines: ", ", ".join(top3),"!")

Try out these top 3 cuisines:  Thai, Japanese, American !
